# 🧪 Backtest Example

Run a strategy through the backtesting engine and inspect the results.

> **Note:** The backtester is not yet implemented (Level 0). This notebook provides the scaffolding and will be updated once the engine is ready.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import matplotlib.pyplot as plt

from backend.etl.data_access.storage.HistoricalDataStore import HistoricalDataStore
from backend.scitus.features import FeatureStore

## 1. Load Data & Features

In [ ]:
store = HistoricalDataStore()
fs = FeatureStore(data_store=store)

ASSET = "BTCUSDT"
START = "2024-01-01"
END   = "2024-12-31"

raw = store.query(asset_id=ASSET, start_date=START, end_date=END)
features = fs.compute(ASSET, ["rsi_14", "sma_20"], start=START, end=END)

print(f"Raw: {len(raw)} rows, Features: {len(features)} rows")

## 2. Run Backtest (placeholder)

Replace this cell with `BacktestEngine` usage once Level 0 is implemented.

In [ ]:
# Placeholder: simple buy-and-hold equity curve
if not raw.empty and 'close' in raw.columns:
    raw = raw.copy()
    raw['returns'] = raw['close'].pct_change()
    raw['equity'] = (1 + raw['returns']).cumprod()
    
    results = raw[['timestamp', 'close', 'returns', 'equity']].dropna()
    results.head(10)
else:
    print("No data loaded. Check HistoricalDataStore configuration.")
    results = pd.DataFrame()

## 3. Equity Curve

In [ ]:
if not results.empty:
    fig, ax = plt.subplots(figsize=(14, 5))
    ax.plot(results['timestamp'], results['equity'], linewidth=1.0)
    ax.set_title(f'{ASSET} Buy & Hold Equity Curve')
    ax.set_ylabel('Equity (normalized)')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No results to plot.")

## 4. Summary Statistics

In [ ]:
if not results.empty:
    total_return = results['equity'].iloc[-1] - 1
    max_drawdown = (results['equity'] / results['equity'].cummax() - 1).min()
    sharpe = results['returns'].mean() / results['returns'].std() * (252 ** 0.5)
    
    summary = pd.DataFrame({
        'Metric': ['Total Return', 'Max Drawdown', 'Sharpe Ratio'],
        'Value': [f'{total_return:.2%}', f'{max_drawdown:.2%}', f'{sharpe:.2f}']
    })
    summary
else:
    print("No results available.")